# Word Embeddings
Use word embeddings to explore the Wall Street Journal from 2019/11 to 2021/11.  
This jupyter notebook is supposed to be run on the ProQues TDM studio.

In [10]:
import dask
import nltk
from gensim import models
import pandas as pd
import dwe # custom module for today

In [11]:
import nltk
nltk.data.path.append('/home/ec2-user/SageMaker/nltk_data/')
from nltk.corpus import stopwords
from nltk.tokenize import punkt

In [3]:
 !conda list | grep "dask"

In [2]:
import dask.dataframe as dd

In [8]:
# The part after this cell should follow the "CovertToDataframeMultiprocessing.ipynb" file.
# But the two lines right below are repeated from "CoverToDataframeMultiprocessing" for illusration purpose.
# Creating a dataframe, setting each of the columns to one of the lists we made in the cell above
df = pd.DataFrame({'Article ID': filename_list,'Text': text_list, 'Date': date_list})

In [9]:
# Showing the dataframe
df

,Article ID,Text,Date
0,2457405332.xml,\nJoe Biden racked up Electoral College votes ...,2020-11-05
1,2384277728.xml,\nDanita Sienknecht was on a car ride with her...,2020-03-30
2,2524054500.xml,\nSimon Heaton said he expected to get a prest...,2021-05-10
3,2325226974.xml,\nHere are some of the companies with shares e...,2019-12-13
4,2599921634.xml,\nNovartis AG bet big on its new cholesterol-b...,2021-11-21
...,...,...,...
96594,2512354895.xml,\n\n\n\n\n\n\n\n\n\n\n\nBiden administration o...,2021-04-14
96595,2577580565.xml,"\nTOKYO—Fumio Kishida, a former foreign minist...",2021-09-30
96596,2443906195.xml,\nU.K. budget carrier EasyJet PLC hired a new ...,2020-09-18
96597,2540658373.xml,"\nMINNEAPOLIS\nDriver Plows Into\nProtest, Kil...",2021-06-15


In [13]:
df['YM'] = df['Date'].str.slice(0,7)

In [14]:
df

,Article ID,Text,Date,YM
0,2457405332.xml,\nJoe Biden racked up Electoral College votes ...,2020-11-05,2020-11
1,2384277728.xml,\nDanita Sienknecht was on a car ride with her...,2020-03-30,2020-03
2,2524054500.xml,\nSimon Heaton said he expected to get a prest...,2021-05-10,2021-05
3,2325226974.xml,\nHere are some of the companies with shares e...,2019-12-13,2019-12
4,2599921634.xml,\nNovartis AG bet big on its new cholesterol-b...,2021-11-21,2021-11
...,...,...,...,...
96594,2512354895.xml,\n\n\n\n\n\n\n\n\n\n\n\nBiden administration o...,2021-04-14,2021-04
96595,2577580565.xml,"\nTOKYO—Fumio Kishida, a former foreign minist...",2021-09-30,2021-09
96596,2443906195.xml,\nU.K. budget carrier EasyJet PLC hired a new ...,2020-09-18,2020-09
96597,2540658373.xml,"\nMINNEAPOLIS\nDriver Plows Into\nProtest, Kil...",2021-06-15,2021-06


In [15]:
df = df.loc[:, ['YM', 'Text']]

In [16]:
# Create dask dataframe and partition them into 8 partitions
df_dask = dd.from_pandas(df,npartitions=8)

In [17]:
df_dask.head()

,YM,Text
0,2020-11,\nJoe Biden racked up Electoral College votes ...
1,2020-03,\nDanita Sienknecht was on a car ride with her...
2,2021-05,\nSimon Heaton said he expected to get a prest...
3,2019-12,\nHere are some of the companies with shares e...
4,2021-11,\nNovartis AG bet big on its new cholesterol-b...


# Tokenize the data

In [18]:
# Tokenize the text using dwe
dict_dask2 = {}
months = df_dask['YM'].unique()
df_group = df_dask.groupby('YM')
for month in months:
    dict_dask2[month] = dwe.tokenize_ob(df_group.get_group(month).Text).compute()

/home/ec2-user/SageMaker/.conda/envs/sample_env/lib/python3.7/site-packages/dask/dataframe/core.py:3718: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Text', 'object'))

  warnings.warn(meta_warning(meta))
/home/ec2-user/SageMaker/.conda/envs/sample_env/lib/python3.7/site-packages/dask/dataframe/core.py:3718: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)

/home/ec2-user/SageMaker/.conda/envs/sample_env/lib/python3.7/site-packages/dask/dataframe/core.py:3718: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('Text', 'object'))

  warnings.warn(meta_warning(meta))
/home/ec2-user/SageMaker/.conda/envs/sample_env/lib/python3.7/site-packages/dask/dataframe/core.py:3718: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)

In [27]:
dict_dask2['2021-11'].head(20)

4      [novarti, ag, bet, big, new, cholesterolbust, ...
13     [us, labor, market, sprang, back, life, octob,...
28     [least, peopl, die, two, dozen, other, treat, ...
42     [grizzli, flat, califdon, lee, wife, amanda, m...
58     [madonna, public, critic, instagram, compani, ...
76     [franc, take, european, council, presid, janua...
79     [poor, harvest, spring, wheat, concern, winter...
156    [washington, first, volley, nation, footbal, l...
159    [procter, gambl, co, recal, old, spice, secret...
164    [known, firearmsthem, brand, support, militari...
177    [tokyo, nissan, motor, co, said, monday, plan,...
197    [diana, gabaldon, author, best, known, outland...
204    [two, payment, option, vy, hard, shopper, atte...
225    [midlothian, texa, last, winter, peopl, costum...
228    [sen, ron, wyden, propos, billionair, tax, acc...
239    [pfizer, inc, said, thursday, agre, billion, d...
251    [washington, presid, biden, host, ceremoni, mo...
288    [extrem, disloc, market,

# Aligning Diachronic Word Embeddings

Use Gensim's Word2Vec implementation to transform words into vectors.

In [22]:
model = {}
for month, tokens in dict_dask2.items():
    model[month] = models.word2vec.Word2Vec(tokens, workers=-1)
    model[month].save('w2v_{}.model'.format(month))

model = {str(month): models.KeyedVectors.load('w2v_{}.model'.format(month)) for month in months}

In [24]:
# This is the vector representation of a word:
model['2021-09'].wv['good'] - model['2021-09'].wv['bad']

array([-0.01246194, -0.00423844, -0.00754492, -0.00094433, -0.0076013 ,
       -0.0039413 ,  0.00354507,  0.00443552, -0.0053397 , -0.00448217,
        0.00812644, -0.01373298,  0.00528097, -0.00261526, -0.00158261,
        0.00662444,  0.00707565, -0.00618023,  0.00849694, -0.00534548,
        0.00836258, -0.00739764,  0.00376004,  0.00388304, -0.0100096 ,
        0.00204969,  0.00824579, -0.00489771,  0.01671576,  0.00554534,
       -0.00456122, -0.00910323,  0.01165122,  0.00956067,  0.00326219,
       -0.01616808,  0.0046078 , -0.0040546 , -0.00817754, -0.01651328,
        0.00313228,  0.00101648,  0.00323665,  0.01068779,  0.01575492,
        0.00822953,  0.00199978,  0.00343103, -0.007583  ,  0.00367907,
        0.00315927,  0.00935874, -0.00433332,  0.00300984, -0.00906182,
        0.00901577, -0.01358785,  0.00736532, -0.01367095, -0.01630726,
       -0.01458818,  0.0046094 ,  0.0012912 ,  0.00343539, -0.00633125,
        0.013795  ,  0.00129037,  0.00793879, -0.00089897,  0.01

In [31]:
model['2021-09'].wv['certain']-model['2021-09'].wv['uncertain']

array([ 0.00238759, -0.00905487,  0.01684976, -0.00259188,  0.0003051 ,
        0.00352847, -0.00081572,  0.00749601,  0.0084063 ,  0.01165399,
        0.00863019,  0.01082332, -0.01708635, -0.00474653,  0.00330676,
        0.00573033, -0.00324709, -0.00581241,  0.00340554, -0.00061988,
        0.00110295,  0.00274357, -0.00974507,  0.00344604, -0.00409065,
        0.00578468, -0.00585447, -0.00065301,  0.00258926,  0.00149495,
        0.00045994,  0.01031436,  0.01083726, -0.00607581,  0.01331528,
        0.01213955, -0.01037435,  0.00162626,  0.00698686,  0.00200312,
       -0.01169665,  0.00290381, -0.00250407, -0.00675524, -0.00913734,
        0.0042626 ,  0.00784279, -0.00172653, -0.00687744, -0.00050499,
        0.00071931,  0.01472743,  0.01618569,  0.00239614,  0.00531416,
       -0.01645597,  0.0083224 , -0.00171803, -0.01376249, -0.00318439,
       -0.01074727, -0.00494634, -0.00136675,  0.01503177, -0.0109706 ,
       -0.01716124, -0.00405496,  0.01767059,  0.0030091 ,  0.00

In [47]:
print(model['2021-09'].wv.most_similar('good'))

[('bronx', 0.36144378781318665), ('auto', 0.3456558287143707), ('studentloan', 0.3406968116760254), ('heartbreak', 0.332455575466156), ('dens', 0.3287903666496277), ('cent', 0.32342860102653503), ('geometr', 0.3194636106491089), ('shanksvill', 0.3189867436885834), ('margaret', 0.31608015298843384), ('kenneth', 0.31579887866973877)]


In [48]:
print(model['2021-09'].wv.most_similar('bad'))

[('rational', 0.41860443353652954), ('widest', 0.35977911949157715), ('brake', 0.3488847315311432), ('soot', 0.32659947872161865), ('venkatprwsjcom', 0.32314053177833557), ('iea', 0.31359636783599854), ('rail', 0.30930137634277344), ('retent', 0.3029954433441162), ('broken', 0.3028797507286072), ('charley', 0.30199503898620605)]


In [49]:
print(model['2021-10'].wv.most_similar('certain'))

[('parisroubaix', 0.3387027084827423), ('kyrsten', 0.3372078537940979), ('cf', 0.33583611249923706), ('gamechang', 0.3256683647632599), ('martyn', 0.3209635615348816), ('aniston', 0.3203233778476715), ('amount', 0.3187102675437927), ('oxfam', 0.31554776430130005), ('coup', 0.3130550980567932), ('humana', 0.3085482120513916)]


In [50]:
print(model['2021-10'].wv.most_similar('uncertain'))

[('amish', 0.3379340171813965), ('worldview', 0.33166059851646423), ('smooth', 0.3286925256252289), ('invers', 0.3266356289386749), ('propos', 0.3259832561016083), ('homerenov', 0.3255958557128906), ('speaker', 0.31199002265930176), ('cohort', 0.3119129240512848), ('oil', 0.3117658495903015), ('diablo', 0.3100152611732483)]


In [60]:
# assign two models
dwe.smart_procrustes_align_gensim(model['2020-11'], model['2021-11'])

Alignment Complete


In [61]:
print('2020-11', model['2020-11'].wv.distance('good', 'certain'))
print('2021-11', model['2021-11'].wv.distance('good', 'certain'))

2020-11 1.1797177493572235
2021-11 0.9868703782558441


In [64]:
dwe.smart_procrustes_align_gensim(model['2019-12'], model['2021-08'])

Alignment Complete


In [65]:
print('2019-12', model['2019-12'].wv.distance('good', 'certain'))
print('2021-08', model['2021-08'].wv.distance('good', 'certain'))

2019-12 1.1711811125278473
2021-08 0.9483139030635357
